# Christofides Algorithm for Network Design

In [1]:
#Load the distance_matrix.xlsx
from pandas import read_excel
file_name='C:\\Users\\zxy61\\OneDrive\\OneCareer\\DHL Transportation Modeling\\Distribution Case Study\\distance_matrix.xlsx'
data=read_excel(file_name,index_col=0)

In [4]:
def tsp(data):
    # build a graph
    G = build_graph(data)
    print("Graph: ", G)

    # build a minimum spanning tree
    MSTree = minimum_spanning_tree(G)
    print("MSTree: ", MSTree)

    # find odd vertexes
    odd_vertexes = find_odd_vertexes(MSTree)
    print("Odd vertexes in MSTree: ", odd_vertexes)

    # add minimum weight matching edges to MST
    minimum_weight_matching(MSTree, G, odd_vertexes)
    #print("Minimum weight matching: ", MSTree)

    # find an eulerian tour
    eulerian_tour = find_eulerian_tour(MSTree, G)

    #print("Eulerian tour: ", eulerian_tour)

    current = eulerian_tour[0]
    path = [current]
    visited = [False] * len(eulerian_tour)
    visited[0] = True

    length = 0

    for v in eulerian_tour[1:]:
        if not visited[v]:
            path.append(v)
            visited[v] = True

            length += G[current][v]
            current = v

    path.append(path[0])

    print("result path: ",path)
    print("Result length of the path: ", round(length,4),'miles')

    #return length, path



def build_graph(data):
    graph={}
    for i in range(len(data)):
        for j in range(len(data)):
            if i != j:
                if i not in graph:
                    graph[i]={}

                graph[i][j]=data['X%s' %(j+100)].loc[(i+100)]
    return graph


class UnionFind:
    def __init__(self):
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root

    def __iter__(self):
        return iter(self.parents)

    def union(self, *objects):
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r], r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest


def minimum_spanning_tree(G):
    tree = []
    subtrees = UnionFind()
    for W, u, v in sorted((G[u][v], u, v) for u in G for v in G[u]):
        if subtrees[u] != subtrees[v]:
            tree.append((u, v, W))
            subtrees.union(u, v)

    return tree


def find_odd_vertexes(MST):
    tmp_g = {}
    vertexes = []
    for edge in MST:
        if edge[0] not in tmp_g:
            tmp_g[edge[0]] = 0

        if edge[1] not in tmp_g:
            tmp_g[edge[1]] = 0

        tmp_g[edge[0]] += 1
        tmp_g[edge[1]] += 1

    for vertex in tmp_g:
        if tmp_g[vertex] % 2 == 1:
            vertexes.append(vertex)

    return vertexes


def minimum_weight_matching(MST, G, odd_vert):
    vertices = []
    for W, u, v in sorted((G[u][v], u, v)
                          for u in G
                          for v in G[u]
                          if v in odd_vert and u in odd_vert):
        if u not in vertices and v not in vertices:
            length = G[v][u]
            MST.append((u, v, length))
            vertices.append(u)
            vertices.append(v)


def find_eulerian_tour(MatchedMSTree, G):
    # find neigbours
    neighbours = {}
    for edge in MatchedMSTree:
        if edge[0] not in neighbours:
            neighbours[edge[0]] = []

        if edge[1] not in neighbours:
            neighbours[edge[1]] = []

        neighbours[edge[0]].append(edge[1])
        neighbours[edge[1]].append(edge[0])

    # print("Neighbours: ", neighbours)

    # finds the hamiltonian circuit
    start_vertex = MatchedMSTree[0][0]
    EP = [neighbours[start_vertex][0]]

    while len(MatchedMSTree) > 0:
        for i, v in enumerate(EP):
            if len(neighbours[v]) > 0:
                break

        while len(neighbours[v]) > 0:
            w = neighbours[v][0]

            remove_edge_from_matchedMST(MatchedMSTree, v, w)

            del neighbours[v][(neighbours[v].index(w))]
            del neighbours[w][(neighbours[w].index(v))]

            i += 1
            EP.insert(i, w)

            v = w

    return EP


def remove_edge_from_matchedMST(MatchedMST, v1, v2):

    for i, item in enumerate(MatchedMST):
        if (item[0] == v2 and item[1] == v1) or (item[0] == v1 and item[1] == v2):
            del MatchedMST[i]

    return MatchedMST


In [5]:
tsp(data)

Graph:  {0: {1: 92.5375, 2: 91.9875, 3: 54.17499999999999, 4: 70.76666666666665, 5: 35.65833333333334, 6: 99.96249999999999, 7: 100.375, 8: 93.59166666666667, 9: 92.03333333333333, 10: 42.395833333333336, 11: 85.89166666666667, 12: 78.55833333333332, 13: 124.89583333333334, 14: 80.89583333333334, 15: 94.50833333333333, 16: 58.34583333333333, 17: 93.6375, 18: 47.11666666666667, 19: 94.82916666666667, 20: 48.8125, 21: 142.175, 22: 94.69166666666666, 23: 79.93333333333332, 24: 22.458333333333332, 25: 36.025, 26: 13.108333333333334, 27: 138.6916666666667, 28: 67.14583333333334, 29: 83.96666666666667, 30: 99.22916666666667, 31: 152.4875, 32: 70.44583333333333, 33: 88.41249999999998, 34: 94.69166666666668, 35: 88.73333333333333, 36: 93.6375, 37: 79.6125, 38: 85.75416666666668, 39: 78.87916666666666, 40: 55.77916666666667, 41: 62.791666666666664, 42: 105.82916666666667, 43: 124.43750000000001, 44: 80.80416666666667, 45: 91.80416666666667, 46: 102.85000000000001, 47: 77.1375, 48: 58.2999999999

In [17]:
# The optimized route start and end with WH (60)
result_path=[60, 28, 29, 9, 17, 30, 6, 46, 13, 31, 21, 27, 55, 
             45, 35, 1, 14, 47, 23, 37, 2, 57, 15, 8, 22, 49, 7, 
             54, 36, 34, 42, 58, 43, 33, 11, 53, 19, 4, 44, 59, 16, 
             3, 40, 56, 41, 18, 51, 20, 5, 25, 52, 38, 48, 26, 0, 24, 
             10, 50, 32, 12, 39, 60]